In [9]:
# trying to use data_viz1 for this

import dill
import mdtraj as md
import numpy as np
import nglview as nv
import torch
import io

In [10]:
import dill
import torch

relevant_pkl = "/home/bfd21/rds/hpc-work/tbg/cyclization/jobs/l1-sample-Feb-19/traj-rec_batch_3.pkl"

# Read raw pickle contents safely
with open(relevant_pkl, "rb") as f:
    raw_data = dill.load(f)

print(f"Data type: {type(raw_data)}")

# Check for any remaining CUDA tensors
def check_for_cuda(obj):
    if isinstance(obj, torch.Tensor):
        print(f"Found tensor on CUDA? {obj.is_cuda}")
    elif isinstance(obj, list):
        for i, item in enumerate(obj):
            print(f"Checking list item {i}")
            check_for_cuda(item)
    elif isinstance(obj, dict):
        for key, value in obj.items():
            print(f"Checking dict key {key}")
            check_for_cuda(value)

check_for_cuda(raw_data)


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
# Path to the pickle file
relevant_pkl = "/home/bfd21/rds/hpc-work/tbg/cyclization/jobs/l1-sample-Feb-19/traj-rec_batch_3.pkl"

# Define a safe loader function that forces tensors to CPU
def safe_torch_load(f, **kwargs):
    return torch.load(f, map_location="cpu", **kwargs)

# Read the pickle file
with open(relevant_pkl, "rb") as f:
    raw_data = f.read()  # Read raw bytes first

# Try loading with `dill`, ensuring that tensors go to CPU
with io.BytesIO(raw_data) as f:
    data = dill.load(f, ignore=True)  # Ensure `ignore=True` to avoid unnecessary errors

# If there are any torch tensors inside, convert them to numpy arrays
def move_to_cpu(obj):
    if isinstance(obj, torch.Tensor):
        return obj.cpu().numpy()  # Convert tensor to numpy
    elif isinstance(obj, dict):
        return {key: move_to_cpu(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [move_to_cpu(value) for value in obj]
    else:
        return obj

data = move_to_cpu(data)

# Verify loaded data
print(type(data))
if isinstance(data, dict):
    print("Keys:", data.keys())
    print(data.keys())

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
relevant_pkl = "/home/bfd21/rds/hpc-work/tbg/cyclization/jobs/l1-sample-Feb-19/traj-rec_batch_3.pkl"

# Load the positions and times
with open(relevant_pkl, "rb") as f:
    data = dill.load(f)

# Assuming data is a dictionary with 'positions' as an array of shape (n_frames, n_atoms, 3)
# and 'times' as an array of shape (n_frames,)
positions = np.array(data["positions"])  # Shape: (n_frames, n_atoms, 3)
times = np.array(data["times"])  # Shape: (n_frames,)

# Load the PDB topology
topology = md.load("your_structure.pdb").topology

# Create the trajectory
traj = md.Trajectory(positions, topology)

# Create an nglview widget
view = nv.show_mdtraj(traj)

# Display the viewer
view